In [30]:
import requests
import json
import warnings
import time
import pandas as pd
import numpy as np
warnings.filterwarnings("ignore")

In [32]:
url = 'https://www.amap.com/service/poiInfo'
payload = {
    'query_type': 'TQUERY',
    'pagesize': 20,
    'pagenum': 1,
    'qii': True,
    'cluster_state': 5,
    'need_utd': True,
    'utd_sceneid': 1000,
    'div': 'PC1000',
    'addr_poi_merge': True,
    'is_classify': True,
    'zoom': 17,
    'city': 340100,
    'geoobj': '116.951392|31.591949|117.533668|32.077299',
    'keywords': '丁香家园一期'
}

headers = {
    'Host': 'www.amap.com',
    'amapuuid':'bdfc589d-8c81-49da-99dd-06500b389b02',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Referer': 'https://www.amap.com',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'x-csrf-token': None,
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5'
}

cookies = {
    'l':'dBaBgHluqWhTnkiQBOfwRurza77O5COj5sPzaNbMiICPOD7d4jTNWZKSrmO9CnGVH6zko3zIKqC8BWLHNPl8Z1u2AC1hE_iindC..',
    'isg':'BN_f1FuZ_nPvZPpThpWQExdxbjVpRDPmTfooqXEp-w7VAOWCdxXwNn4SwtDbmAte',
    'CNZZDATA1255626299':'728144284-1575339666-|1575339666',
}

In [43]:
df = pd.read_csv('../data/合肥_20191126.csv')
names = list(np.unique(df['address'].values))

aoi_set = []

In [ ]:
# names = ['宝利丰广场']
while len(names) > 0:
    payload['keywords'] = names.pop(0)
    print(payload['keywords'])
    
    r = requests.get(url, params=payload, headers=headers, cookies=cookies)
    content = r.text

    o = json.loads(content)
    if o['status'] == '1':
        if 'poi_list' in o['data'].keys():
            for poi in o['data']['poi_list']:
                lock = False
                for item in aoi_set:
                    if item['name'] == poi['name']:
                        lock = True
                        break

                if lock:
                    continue

                aoi = None
                for i in poi['domain_list']:
                    if i['id'] == '1013' and 'value' in i.keys():
                        aoi = i['value']

                if aoi is not None:
                    one = {
                        'shape': aoi,
                        'name': poi['name'],
                        'lng':  poi['longitude'],
                        'lat': poi['latitude']
                    }
                    print(one)
                    aoi_set.append(one)
                
    else:
        print('reflection')
        break
    
    time.sleep(5)

In [46]:
aoi_df = pd.DataFrame(aoi_set)

In [48]:
aoi_df.to_csv('../hf_poi.csv', index=False)